In [4]:
!pip install langchain chromadb groq python-dotenv sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 122.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 98.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 124.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.7/517.7 kB 45.8 MB/s

In [5]:
!pip install pypdf langchain


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 9.8 MB/s eta 0:00:00


In [2]:
import os
api_key = os.environ["GROQ_API_KEY"]

KeyError: 'GROQ_API_KEY'

In [21]:
import os
from google.colab import userdata

# Access the GROQ_API_KEY from Colab secrets
groq_api_key = userdata.get("GROQ_API_KEY")

# You can now use groq_api_key in your code, for example:
# client = Groq(api_key=groq_api_key)
# or set it as an environment variable if a library expects it
# os.environ["GROQ_API_KEY"] = groq_api_key

print("Groq API key loaded successfully!")

Groq API key loaded successfully!


In [7]:
from google.colab import files

uploaded_files = files.upload()  # Choose your PDFs
pdf_paths = list(uploaded_files.keys())
print(" Uploaded PDFs:", pdf_paths)


Saving china & japan.pdf to china & japan.pdf
 Uploaded PDFs: ['china & japan.pdf']


load and extract text


In [8]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [9]:
from langchain_community.document_loaders import PyPDFLoader

all_docs = []

for path in pdf_paths:
    loader = PyPDFLoader(path)
    docs = loader.load()
    all_docs.extend(docs)

print(f" Loaded {len(all_docs)} pages from {len(pdf_paths)} file(s).")

 Loaded 77 pages from 1 file(s).


split text to chunck

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs_chunks = splitter.split_documents(all_docs)

print(f" Total text chunks created: {len(docs_chunks)}")


 Total text chunks created: 77


In [13]:
from langchain_community.vectorstores import Chroma
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vectordb = Chroma.from_documents(docs_chunks, embedding=embeddings, persist_directory="./chroma_db")
vectordb.persist()

print(" Vector database created and persisted!")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

 Vector database created and persisted!


/tmp/ipython-input-4246353149.py:7: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [12]:
!pip install -U langchain-huggingface

In [22]:
# Replace this with your question
question = "What is the main topic of the document?"
answer = rag_query(question)
print(" Answer:\n", answer)

NameError: name 'rag_query' is not defined

In [23]:
from groq import Groq
import os
from google.colab import userdata

# Ensure the API key is loaded from secrets
groq_api_key = userdata.get("GROQ_API_KEY")

# Define the rag_query function
def rag_query(question, k=3):
    """Retrieve relevant context from PDFs and ask Groq model."""
    # Check if the API key is available
    if not groq_api_key:
        return "Error: GROQ_API_KEY not found in Colab secrets."

    client = Groq(api_key=groq_api_key)

    docs = vectordb.similarity_search(question, k=k)
    context = "\n\n".join([d.page_content for d in docs])

    prompt = f"""
    You are a helpful AI assistant. Use the context below to answer the user's question clearly and accurately.

    Context:
    {context}

    Question: {question}
    """

    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[{"role": "user", "content": prompt}],
    )

    return response.choices[0].message.content

# Now call the rag_query function
question = "What is the main topic of the document?"
answer = rag_query(question)
print(" Answer:\n", answer)

 Answer:
 The main topic of the document appears to be an overview of significant geographical features, economic industries, and administrative divisions in China and possibly Japan, although only a few Japanese locations are specifically mentioned.


In [25]:
from groq import Groq
import os
from google.colab import userdata

# Access the GROQ_API_KEY from Colab secrets
groq_api_key = userdata.get("GROQ_API_KEY")

# Define the rag_query function
def rag_query(question, k=3):
    """Retrieve relevant context from PDFs and ask Groq model."""
    # Check if the API key is available
    if not groq_api_key:
        return "Error: GROQ_API_KEY not found in Colab secrets."

    client = Groq(api_key=groq_api_key)

    docs = vectordb.similarity_search(question, k=k)
    context = "\n\n".join([d.page_content for d in docs])

    prompt = f"""
    You are a helpful AI assistant. Use the context below to answer the user's question clearly and accurately.

    Context:
    {context}

    Question: {question}
    """

    response = client.chat.completions.create(
        model="llama-3.1-8b-instant", # Changed model to an available one
        messages=[{"role": "user", "content": prompt}],
    )

    return response.choices[0].message.content

# Initialize the Groq client outside the function if needed for other purposes
# client = Groq(api_key=groq_api_key)

In [26]:
question = "Summarize the main topic of the uploaded document."
answer = rag_query(question)
print(" Answer:\n", answer)


 Answer:
 Based on the provided context, it appears that the main topic of the uploaded document is a comparison or description of various regions in Japan, China, and other parts of Asia.

Specifically, it seems to be discussing the following regions:

1. A region in western Japan involving Hiroshima, Ube, Kyushu Plain, Kyushu cities, etc.
2. A region in central Japan involving Nagoya, Gifu, Hamamatsu, etc.
3. A region in China involving the Tarim Basin, Tarim river, Lop Nur lake, etc.

However, without the actual uploaded document, it's difficult to provide a detailed summary. If you provide the document, I'd be happy to help.


In [27]:
question = "Explain about china geography."
answer = rag_query(question)
print(" Answer:\n", answer)

 Answer:
 China, the world's third-largest country by land area, boasts a diverse and vast geography that encompasses numerous mountains, plateaus, deserts, and rivers. Here's an overview of China's geography:

**Geographical Regions:**

1. **Eastern China**: This region includes the Shandong Peninsula, the Yellow Sea coast, and the Yangtze River Delta, which is the country's most populous and economically developed area.
2. **Northern China**: This region encompasses the Great Khingan Range, the Inner Mongolian Plateau, and the Ordos Desert.
3. **Southern China**: This region includes the Yunnan Plateau, the Sichuan Basin, and the subtropical regions surrounding the Pearl River Delta.
4. **Western China**: This region comprises the Tibetan Plateau, the Qilian Mountains, and the Gansu Corridor.

**Mountain Ranges:**

1. **Tibetan Plateau**: The world's highest plateau, known as the "Roof of the World," stretching over 2.5 million square kilometers across western China.
2. **Nan Shan**:

In [28]:
question = "Explain about America geography."
answer = rag_query(question)
print(" Answer:\n", answer)

 Answer:
 Unfortunately, I don't see any information about American geography provided in the context. However, based on my general knowledge, I can provide you with an overview of the geography of the United States of America.

The geography of America is diverse and vast. Some notable features include:

1. **Mountain ranges:** The Rocky Mountains, the Appalachian Mountains, the Sierra Nevada, and the Cascades are just a few examples.
2. **Rivers:** The Mississippi River, the Colorado River, and the Rio Grande are some of the longest and most significant rivers in the country.
3. **Coastlines:** The United States has three main coastlines: the Atlantic, the Pacific, and the Gulf of Mexico Coast.
4. **States with unique geography:** Hawaii is an archipelago, Alaska is vast and sparsely populated, and Florida has a large subtropical region.

Additionally, some notable geographical regions in the United States include:

* The Great Plains, which cover much of the Midwest.
* The Great Lak

In [29]:
question = "Expalinth Japan disadvantage in geography ."
answer = rag_query(question)
print(" Answer:\n", answer)

 Answer:
 Based on the given context about Japan's physical geography, its disadvantages can be summarized as follows:

1. **Limited Arable Land**: Japan's rugged topography (80% of the country is mountainous) makes it difficult to cultivate the land for agriculture. This limits the amount of arable land available, which can lead to food shortages and dependence on imports.

2. **Small and Swift Rivers**: Japan's small and swift-river system leads to flash flooding and limits the potential for irrigation and water storage. This makes it challenging to control floods, meet the water needs of the population, and support agriculture.

3. **No Large River System**: The absence of a large river system in Japan hinders the transportation of goods and people, which can impact economic development. Rivers often play a crucial role in shaping trade routes and connecting different regions of a country.

4. **Limited Natural Resources**: Japan's geography also means that it lacks large natural re